In [ ]:
# 读取数据时候可以用Multiprocessing
# 能够减少数据读取消耗时间
# import multiprocessing
# with multiprocessing.Pool() as pool:
#     test_identity, test_transaction, train_identity, train_transaction = pool.map(load_data, files)

In [ ]:
# 根据数据类型的最大值和最小值，我们可以把数据类型改成int8,int16,int32,float32这些
# 比如有的数据都是整数，用int就能节省空间
def reduce_mem_usage1(props):
    # 以下代码补充缺失值为 min-1     start_mem_usg = props.memory_usage().sum() / 1024**2
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in.     for col in props.columns:
    if props[col].dtype != object:  # Exclude strings 
        # Print current column type             print("******************************")
        print("Column: ",col)
        print("dtype before: ",props[col].dtype)

        # make variables for Int, max and min             IsInt = False
        mx = props[col].max()
        mn = props[col].min()

        # Integer does not support NA, therefore, NA needs to be filled             if not np.isfinite(props[col]).all():
        NAlist.append(col)
        props[col].fillna(mn-1,inplace=True)

        # test if column can be converted to an integer             asint = props[col].fillna(0).astype(np.int64)
        result = (props[col] - asint)
        result = result.sum()
        if result > -0.01 and result < 0.01:
            IsInt = True


        # Make Integer/unsigned Integer datatypes             if IsInt:
            if mn >= 0:
                if mx < 255:
                    props[col] = props[col].astype(np.uint8)
                elif mx < 65535:
                    props[col] = props[col].astype(np.uint16)
                elif mx < 4294967295:
                    props[col] = props[col].astype(np.uint32)
                else:
                    props[col] = props[col].astype(np.uint64)
            else:
                if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                    props[col] = props[col].astype(np.int8)
                elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                    props[col] = props[col].astype(np.int16)
                elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                    props[col] = props[col].astype(np.int32)
                elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                    props[col] = props[col].astype(np.int64)

        # Make float datatypes 32 bit             else:
            props[col] = props[col].astype(np.float32)

        # Print new column type             print("dtype after: ",props[col].dtype)
        print("******************************")

    # Print final result     print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

# Memory usage of properties dataframe is : 1775.1522827148438 MB # ___MEMORY USAGE AFTER COMPLETION:___ # Memory usage is: 452.7989959716797 MB # This is 25.507614213197968 % of the initial size

In [ ]:
# EDA感觉作用还是很大的
# 对于分类变量看每个类别不同值的数量
# 当然还可以分箱操作，分类变量也可以
# 分箱其实就是类似我们原来的分段讨论，每一个段用一段函数，这样子的话效果更佳

In [ ]:
# 比赛上很多匿名特征，但是有的信息是可以挖掘出来的，看运气

In [ ]:
# Data Relaxation
# 删除训练集中的所有值出现的评论是测试集中的三倍的值，就是这种值和测试集的差异比较大，没办法预测
# 还可以删除出现次数较少的值，或者只在训练集中出现的值

In [ ]:
# 构建UID
# 就是唯一的用户ID
# 比如根据信用卡匹配信息，支付卡相关信息和买方邮件域相同的认为是同一个用户
columns = ['M'+ str(i) for i in range(1,10)]+['card'+ str(i) for i in range(1,7)]+['P_emaildomain']
# identity = pd.concat([train[columns],test[columns]]).drop_duplicates() 
identity = test[columns].drop_duplicates()
identity['identity_id'] = list(range(len(identity)))

all_data = pd.concat([train.drop('isFraud', axis=1), test])
all_data = all_data.merge(identity, on=columns, how='left')
all_data = all_data[~pd.isnull(all_data['identity_id'])]

all_data = all_data.groupby('identity_id')['TransactionDT'].agg(['max','min','count']).reset_index()
all_data = all_data.sort_values('min',ascending=True).reset_index(drop=True)

all_data['percent'] = 1/len(all_data)
all_data['percent'] = all_data['percent'].cumsum()

all_data[all_data['min']>max(train['TransactionDT'])].shape[0]/all_data.shape[0]
# 用UID对特征进行聚合，但是不用这个作为特征

In [ ]:
# 对抗验证–adversarial validation
# 测试集中很多数据肯定是没有出现在训练集中的，所以用这个来判断哪些列可以确定UID
# 合并训练集和测试集，并且将训练集和测试集的标签分别设置为0和1；
# 构建一个分类器，用于学习the different between testing and training data；
# 找到训练集中与测试集最相似的样本（most resemble data），作为验证集，其余的作为训练集；【也可将概率作为样本的权重，做有权重的交叉验证】
# 观察AUC，理想的状况是在0.5左右。
from xgboost import XGBClassifier
import catboost as cbt
from sklearn.preprocessing import LabelEncoder

for i in ['D' + str(i) for i in range(1,16)]:
    train[i] = np.floor(train.TransactionDT / (24*60*60)) - train[i]
    test[i] = np.floor(test.TransactionDT / (24*60*60)) - test[i]

    features = ['TransactionAmt',
           'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
           'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
           'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
           'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
           'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
           'M5', 'M6', 'M7', 'M8', 'M9']
    all_data = pd.concat([train[features].sample(frac=0.1),
               test[features].sample(frac=0.1)])
    all_data['is_this_transaction_in_test_data'] = [0]*train.sample(frac=0.1).shape[0] + [1]*test.sample(frac=0.1).shape[0]

cat_col = all_data.select_dtypes(object).columns
for i in cat_col:
    lbl = LabelEncoder()
    all_data[i] = lbl.fit_transform(all_data[i].astype(str))

cat_list = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain', 'ProductCD', 'R_emaildomain', 'card4', 'card6']
cbt_model = cbt.CatBoostClassifier(iterations=1000,learning_rate=0.1,verbose=100,eval_metric='AUC')
cbt_model.fit(all_data.drop(['is_this_transaction_in_test_data'],axis=1),all_data['is_this_transaction_in_test_data'])

from sklearn.metrics import roc_auc_score
y_test = cbt_model.predict(all_data.drop(['is_this_transaction_in_test_data'],axis=1))
roc_auc_score(y_test, all_data['is_this_transaction_in_test_data'])

feature = pd.DataFrame({'importance':cbt_model.feature_importances_, 'feature':cbt_model.feature_names_})
feature = feature.sort_values('importance',ascending=False)
feature = feature[feature['importance']!=0]
plt.figure(figsize=(10, 15))
plt.barh(feature['feature'],feature['importance'],height =0.5)
# 就是画出来看哪两个列相等

In [ ]:
features = ['card1', 'addr1', 'D1']
all_data = pd.concat([train[features].sample(frac=0.1),
           test[features].sample(frac=0.1)])
all_data['is_this_transaction_in_test_data'] = [0]*train.sample(frac=0.1).shape[0] + [1]*test.sample(frac=0.1).shape[0]

cat_col = all_data.select_dtypes(object).columns
for i in cat_col:
    lbl = LabelEncoder()
    all_data[i] = lbl.fit_transform(all_data[i].astype(str))

cat_list = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain', 'ProductCD', 'R_emaildomain', 'card4', 'card6']
cbt_model = cbt.CatBoostClassifier(iterations=1000,learning_rate=0.1,verbose=100,eval_metric='AUC')
cbt_model.fit(all_data.drop(['is_this_transaction_in_test_data'],axis=1),all_data['is_this_transaction_in_test_data'])

y_test = cbt_model.predict(all_data.drop(['is_this_transaction_in_test_data'],axis=1))
roc_auc_score(y_test, all_data['is_this_transaction_in_test_data'])

In [ ]:
# Encoding Functions
# FREQUENCY ENCODE TOGETHER def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')

# LABEL ENCODE def encode_LE(col,train=X_train,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True) # Encode the object as an enumerated type or categorical variable.                                              # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html     nm = col
    if df_comb.max()>32000:
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')

# GROUP AGGREGATION MEAN AND STD def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test,
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS     for main_column in main_columns:
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')

                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)

                print("'"+new_col_name+"'",', ',end='')

# COMBINE FEATURES def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str)
    encode_LE(nm,verbose=False)
    print(nm,', ',end='')

# GROUP AGGREGATION NUNIQUE def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

In [ ]:
# The procedure for engineering features is as follows. First you think of an idea and create a new feature. 
# Then you add it to your model and evaluate whether local validation AUC increases or decreases. 
# If AUC increases keep the feature, otherwise discard the feature.
# TRANSACTION AMT CENTS # 有很多小数点可能是海淘（汇率转换） X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32')
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')
print('cents, ', end='')

# FREQUENCY ENCODE: ADDR1, CARD1, CARD2, CARD3, P_EMAILDOMAIN encode_FE(X_train,X_test,['addr1','card1','card2','card3','P_emaildomain'])

# COMBINE COLUMNS CARD1+ADDR1, CARD1+ADDR1+P_EMAILDOMAIN encode_CB('card1','addr1')
encode_CB('card1_addr1','P_emaildomain')

# FREQUENCY ENOCDE encode_FE(X_train,X_test,['card1_addr1','card1_addr1_P_emaildomain'])

# GROUP AGGREGATE encode_AG(['TransactionAmt','D9','D11'],['card1','card1_addr1','card1_addr1_P_emaildomain'],['mean','std'],usena=True)

In [ ]:
# 选择feature，一般也就是一个个加或者一个个删
# 还有就是importance，相关度，时间连续，训练集和测试集分布

# permutation importance：
# 首先我们有一个已经训练好的模型以及该模型的预测表现（如RMSE），
# 比如房价预测模型本来在validation数据上的RMSE是200。然后针对其中的变量（如面积），把这个变量的值全部打乱重新排序，
# 用这个重新排序过的数据来做预测，得到一个预测表现。比如说这下RMSE变成了500，那么面积这个变量的重要性就可以记为500-200=300
# 这个就是针对单个特征来试

# client consistency：
# 若样本1与样本2属于不同的分类，但在特征A、 B上的取值完全一样，那么特征子集{A，B}不应该选作最终的特征集。

In [ ]:
# 时间连续：对于每个特征构建一个模型，用第一个月数据训练，预测最后一个月数据

In [ ]:
# Covariate Shift协变量转换：检查特征在训练集合测试集的分布是否一致，AUC为0.5的时候，说明分布变化不大
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import gc

def covariate_shift(feature):
    df_train = pd.DataFrame(data={feature: train[feature], 'isTest': 0})
    df_test = pd.DataFrame(data={feature: test[feature], 'isTest': 1})

    # Creating a single dataframe     df = pd.concat([df_train, df_test], ignore_index=True)

    # Encoding if feature is categorical     if str(df[feature].dtype) in ['object', 'category']:
        df[feature] = LabelEncoder().fit_transform(df[feature].astype(str))

    # Splitting it to a training and testing set     X_train, X_test, y_train, y_test = train_test_split(df[feature], df['isTest'], test_size=0.33, random_state=47, stratify=df['isTest'])

    clf = lgb.LGBMClassifier(**params, num_boost_round=500)
    clf.fit(X_train.values.reshape(-1, 1), y_train)
    roc_auc =  roc_auc_score(y_test, clf.predict_proba(X_test.values.reshape(-1, 1))[:, 1])

    del df, X_train, y_train, X_test, y_test
    gc.collect();

    return roc_auc
    # 如果输出比较高，就需要删除一部分数据了

In [ ]:
# NAN search
# 根据缺失值个数划分block，即当原始特征中缺失值个数都是89个，则它们划分为同一个block
nans_df = train.isna()
nans_groups={}
i_cols = ['V'+str(i) for i in range(1,340)]
for col in train.columns:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
# 每一个block的特征其实是可能有相关性的，有三种方法处理
# Applied PCA on each group individually
# Selected a maximum sized subset of uncorrelated columns from each group
# Replaced the entire group with all columns averaged.

In [ ]:
# 首先根据NaN的数量，D11 & V1-V11分为一个block，接着画出V1-V11相关性图：
def make_corr(Vs,Vtitle=''):
    cols = ['TransactionDT'] + Vs
    plt.figure(figsize=(15,15))
    sns.heatmap(train[cols].corr(), cmap='RdBu_r', annot=True, center=0.0, fmt=".2f")
    if Vtitle!='': plt.title(Vtitle,fontsize=14)
    else: plt.title(Vs[0]+' - '+Vs[-1],fontsize=14)
    plt.show()
# 将相关性大于0.75的看为同一组，即D11 & V1-V11可划分为 [[V1],[V2,V3],[V4,V5],[V6,V7],[V8,V9],[V10,V11]]，
# 对每个group保留nunique值多大的列
grps = [[1],[2,3],[4,5],[6,7],[8,9],[10,11]]
def reduce_group(grps,c='V'):
    use = []
    for g in grps:
        mx = 0
        vx = g[0]
        for gg in g:
            n = train[c+str(gg)].nunique()
            if n>mx:
                mx = n
                vx = gg
        use.append(vx)    # 保留 subset 中 unique 值最多的列     print('Use these',use)
reduce_group(grps)

In [ ]:
# 不要相信单独一个validation，我们可以构造多个validation：使用前四个月训练，跳过一个月，预测最后一个月；前两个月训练，跳过2个月，预测最后一个月；
# 第一个月训练，跳过三个月，预测最后一个月。

In [ ]:
# 为了防止过拟合，不能直接使用 UID，因为在测试集中有 60% 多的新用户。但是可以根据UID来提取特征：
new_features = df.groupby('uid')[columns].agg(['mean'])
# 这样模型就有能力来识别未看过的用户

In [ ]:
# Post processing
# 因为同一个人的所有交易都是isFraud = 0或全部isFraud = 1。
# 换句话说，它们的所有预测都是相同的。因此，我们的后期处理是将一个客户的所有预测替换为它们的平均预测，包括训数据集中的isFraud值。
X_test['isFraud'] = sample_submission.isFraud.values
X_train['isFraud'] = y_train.values
comb = pd.concat([X_train[['isFraud']],X_test[['isFraud']]],axis=0)

uids = pd.read_csv('data/uids_v4_no_multiuid_cleaning..csv',usecols=['TransactionID','uid']).rename({'uid':'uid2'},axis=1)
comb = comb.merge(uids,on='TransactionID',how='left')
mp = comb.groupby('uid2').isFraud.agg(['mean'])
comb.loc[comb.uid2>0,'isFraud'] = comb.loc[comb.uid2>0].uid2.map(mp['mean'])

uids = pd.read_csv('data/uids_v1_no_multiuid_cleaning.csv',usecols=['TransactionID','uid']).rename({'uid':'uid3'},axis=1)
comb = comb.merge(uids,on='TransactionID',how='left')
mp = comb.groupby('uid3').isFraud.agg(['mean'])
comb.loc[comb.uid3>0,'isFraud'] = comb.loc[comb.uid3>0].uid3.map(mp['mean'])

sample_submission.isFraud = comb.iloc[len(X_train):].isFraud.values
sample_submission.to_csv('sub_xgb_96_PP.csv',index=False)